# Bruise Detection using differemt CNN Architectures

This notebook contains comprehensive comparison framework that tests various CNN designs and analyzes their performance for binary bruise detection (bruise vs normal skin).

### Import libraries and setup

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, f1_score
import pandas as pd
from datetime import datetime
import time
import os
from PIL import Image
from sklearn.model_selection import train_test_split

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

### Data Loading and Preprocessing

In [41]:
def load_binary_data(data_dir='dataset/Wound_dataset copy', img_size=(224, 224)):
    images = []
    labels = []
    
    # Process bruise images (positive class)
    bruise_path = os.path.join(data_dir, 'Bruises')
    if os.path.exists(bruise_path):
        for img_name in os.listdir(bruise_path):
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(bruise_path, img_name)
                try:
                    img = Image.open(img_path)
                    img = img.convert('RGB')
                    img = img.resize(img_size)
                    img_array = np.array(img) / 255.0
                    
                    images.append(img_array)
                    labels.append(1)  # 1 for bruise
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    
    # Process normal skin images (negative class)
    normal_path = os.path.join(data_dir, 'Normal')
    if os.path.exists(normal_path):
        for img_name in os.listdir(normal_path):
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(normal_path, img_name)
                try:
                    img = Image.open(img_path)
                    img = img.convert('RGB')
                    img = img.resize(img_size)
                    img_array = np.array(img) / 255.0
                    
                    images.append(img_array)
                    labels.append(0)  # 0 for normal
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    
    return np.array(images), np.array(labels)

In [42]:
# Load the dataset
X, y = load_binary_data()
print("Dataset shape:", X.shape)
print("Number of bruise images:", np.sum(y == 1))
print("Number of normal images:", np.sum(y == 0))

Dataset shape: (442, 224, 224, 3)
Number of bruise images: 242
Number of normal images: 200


### Shallow CNN architectures

In [43]:
def create_shallow_cnn(input_shape, kernel_size=3, use_batch_norm=False, activation='relu'):
    """Shallow CNN with 2 conv layers"""
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # First conv block
        layers.Conv2D(32, (kernel_size, kernel_size), activation=activation, padding='same'),
    ])
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Second conv block
    model.add(layers.Conv2D(64, (kernel_size, kernel_size), activation=activation, padding='same'))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation=activation))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

def create_shallow_cnn_leaky(input_shape, kernel_size=3, use_batch_norm=False):
    """Shallow CNN with LeakyReLU"""
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # First conv block
        layers.Conv2D(32, (kernel_size, kernel_size), padding='same'),
        layers.LeakyReLU(alpha=0.01),
    ])
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Second conv block
    model.add(layers.Conv2D(64, (kernel_size, kernel_size), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.01))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

### Deep CNN architectures

In [44]:
def create_deep_cnn(input_shape, kernel_size=3, use_batch_norm=False, activation='relu'):
    """Deep CNN with 4-5 conv layers"""
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # First conv block
        layers.Conv2D(32, (kernel_size, kernel_size), activation=activation, padding='same'),
    ])
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Second conv block
    model.add(layers.Conv2D(64, (kernel_size, kernel_size), activation=activation, padding='same'))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Third conv block
    model.add(layers.Conv2D(128, (kernel_size, kernel_size), activation=activation, padding='same'))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Fourth conv block
    model.add(layers.Conv2D(256, (kernel_size, kernel_size), activation=activation, padding='same'))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Fifth conv block
    model.add(layers.Conv2D(512, (kernel_size, kernel_size), activation=activation, padding='same'))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.GlobalAveragePooling2D())
    
    # Dense layers
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation=activation))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

def create_deep_cnn_leaky(input_shape, kernel_size=3, use_batch_norm=False):
    """Deep CNN with LeakyReLU"""
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # First conv block
        layers.Conv2D(32, (kernel_size, kernel_size), padding='same'),
        layers.LeakyReLU(alpha=0.01),
    ])
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Second conv block
    model.add(layers.Conv2D(64, (kernel_size, kernel_size), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.01))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Third conv block
    model.add(layers.Conv2D(128, (kernel_size, kernel_size), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.01))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Fourth conv block
    model.add(layers.Conv2D(256, (kernel_size, kernel_size), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.01))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Fifth conv block
    model.add(layers.Conv2D(512, (kernel_size, kernel_size), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.01))
    
    if use_batch_norm:
        model.add(layers.BatchNormalization())
    
    model.add(layers.GlobalAveragePooling2D())
    
    # Dense layers
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

### Training and evaluation functions

In [45]:
def train_and_evaluate_model(model, model_name, X_train, X_val, X_test, y_train, y_val, y_test, 
                            epochs=15, batch_size=32):
    """Train model and collect metrics"""
    print(f"\n{'='*50}")
    print(f"Training: {model_name}")
    print(f"{'='*50}")
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'Precision', 'Recall']
    )
    
    # Display model summary
    print(f"Parameters: {model.count_params():,}")
    
    # Record start time
    start_time = time.time()
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Record training time
    training_time = time.time() - start_time
    
    # Evaluate on test set
    test_loss, test_accuracy, test_precision, test_recall = model.evaluate(
        X_test, y_test, verbose=0
    )
    
    # Make predictions
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Calculate F1 score and AUC
    f1 = f1_score(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    
    # Store results
    result = {
        'Model': model_name,
        'Parameters': model.count_params(),
        'Training_Time': training_time,
        'Test_Accuracy': test_accuracy,
        'Test_Precision': test_precision,
        'Test_Recall': test_recall,
        'F1_Score': f1,
        'AUC': roc_auc,
        'History': history,
        'Predictions': y_pred_prob
    }
    
    print(f"Training Time: {training_time:.2f}s")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {roc_auc:.4f}")
    
    return result

In [46]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

input_shape = X_train[0].shape
print(f"Input shape: {input_shape}")

Input shape: (224, 224, 3)


### Train Shallow CNNs

In [47]:
def train_shallow_cnns(X_train, X_val, X_test, y_train, y_val, y_test, input_shape, results):
    """Train all shallow CNN configurations"""
    print("TRAINING SHALLOW CNN ARCHITECTURES")
    print("="*60)
    
    # Shallow CNN configurations
    shallow_configs = [
        {'kernel': 3, 'batch_norm': False, 'activation': 'relu', 'name': 'shallow_k3_no_bn_relu'},
        {'kernel': 3, 'batch_norm': True, 'activation': 'relu', 'name': 'shallow_k3_bn_relu'},
        {'kernel': 5, 'batch_norm': False, 'activation': 'relu', 'name': 'shallow_k5_no_bn_relu'},
        {'kernel': 5, 'batch_norm': True, 'activation': 'relu', 'name': 'shallow_k5_bn_relu'},
        {'kernel': 3, 'batch_norm': False, 'activation': 'leaky', 'name': 'shallow_k3_no_bn_leaky'},
        {'kernel': 3, 'batch_norm': True, 'activation': 'leaky', 'name': 'shallow_k3_bn_leaky'},
    ]
    
    for config in shallow_configs:
        if config['activation'] == 'leaky':
            model = create_shallow_cnn_leaky(input_shape, config['kernel'], config['batch_norm'])
        else:
            model = create_shallow_cnn(input_shape, config['kernel'], config['batch_norm'], config['activation'])
        
        result = train_and_evaluate_model(model, config['name'], X_train, X_val, X_test, 
                                        y_train, y_val, y_test)
        results.append(result)
    
    return results

### Train Deep CNNs

In [48]:
def train_deep_cnns(X_train, X_val, X_test, y_train, y_val, y_test, input_shape, results):
    """Train all deep CNN configurations"""
    print("\nTRAINING DEEP CNN ARCHITECTURES")
    print("="*60)
    
    # Deep CNN configurations
    deep_configs = [
        {'kernel': 3, 'batch_norm': False, 'activation': 'relu', 'name': 'deep_k3_no_bn_relu'},
        {'kernel': 3, 'batch_norm': True, 'activation': 'relu', 'name': 'deep_k3_bn_relu'},
        {'kernel': 5, 'batch_norm': False, 'activation': 'relu', 'name': 'deep_k5_no_bn_relu'},
        {'kernel': 5, 'batch_norm': True, 'activation': 'relu', 'name': 'deep_k5_bn_relu'},
        {'kernel': 3, 'batch_norm': False, 'activation': 'leaky', 'name': 'deep_k3_no_bn_leaky'},
        {'kernel': 3, 'batch_norm': True, 'activation': 'leaky', 'name': 'deep_k3_bn_leaky'},
    ]
    
    for config in deep_configs:
        if config['activation'] == 'leaky':
            model = create_deep_cnn_leaky(input_shape, config['kernel'], config['batch_norm'])
        else:
            model = create_deep_cnn(input_shape, config['kernel'], config['batch_norm'], config['activation'])
        
        result = train_and_evaluate_model(model, config['name'], X_train, X_val, X_test, 
                                        y_train, y_val, y_test)
        results.append(result)
    
    return results

### Results summary table

In [49]:
def print_results_table(results):
    """Print detailed comparison table"""
    results_df = pd.DataFrame([{
        'Model': r['Model'],
        'Parameters': f"{r['Parameters']:,}",
        'Accuracy': f"{r['Test_Accuracy']:.4f}",
        'Precision': f"{r['Test_Precision']:.4f}",
        'Recall': f"{r['Test_Recall']:.4f}",
        'F1-Score': f"{r['F1_Score']:.4f}",
        'AUC': f"{r['AUC']:.4f}",
        'Train Time (s)': f"{r['Training_Time']:.1f}"
    } for r in results])
    
    print("\n" + "="*120)
    print("DETAILED RESULTS COMPARISON")
    print("="*120)
    print(results_df.to_string(index=False))
    
    # Find best models
    best_accuracy = max(results, key=lambda x: x['Test_Accuracy'])
    best_f1 = max(results, key=lambda x: x['F1_Score'])
    best_auc = max(results, key=lambda x: x['AUC'])
    fastest = min(results, key=lambda x: x['Training_Time'])
    
    print(f"\n{'='*60}")
    print("BEST PERFORMING MODELS")
    print(f"{'='*60}")
    print(f"Best Accuracy: {best_accuracy['Model']} ({best_accuracy['Test_Accuracy']:.4f})")
    print(f"Best F1-Score: {best_f1['Model']} ({best_f1['F1_Score']:.4f})")
    print(f"Best AUC: {best_auc['Model']} ({best_auc['AUC']:.4f})")
    print(f"Fastest Training: {fastest['Model']} ({fastest['Training_Time']:.1f}s)")

### Batch normalization impact analysis

In [50]:
def analyze_batch_norm(results):
    """Analyze batch normalization impact"""
    df = pd.DataFrame(results)
    df['Depth'] = df['Model'].str.split('_').str[0]
    df['Batch_Norm'] = df['Model'].str.contains('_bn_')
    df['Activation'] = df['Model'].str.split('_').str[-1]
    
    print("="*80)
    print("BATCH NORMALIZATION IMPACT ANALYSIS")
    print("="*80)
    
    # Overall impact
    with_bn = df[df['Batch_Norm'] == True]
    without_bn = df[df['Batch_Norm'] == False]
    
    print(f"Overall Impact:")
    print(f"  With BatchNorm - Avg Accuracy: {with_bn['Test_Accuracy'].mean():.4f}")
    print(f"  Without BatchNorm - Avg Accuracy: {without_bn['Test_Accuracy'].mean():.4f}")
    improvement = ((with_bn['Test_Accuracy'].mean() - without_bn['Test_Accuracy'].mean())/without_bn['Test_Accuracy'].mean())*100
    print(f"  Improvement: {improvement:.2f}%")
    
    # Impact by architecture depth
    print(f"\nImpact by Architecture Depth:")
    for depth in ['shallow', 'deep']:
        depth_data = df[df['Depth'] == depth]
        with_bn_depth = depth_data[depth_data['Batch_Norm'] == True]['Test_Accuracy'].mean()
        without_bn_depth = depth_data[depth_data['Batch_Norm'] == False]['Test_Accuracy'].mean()
        improvement = ((with_bn_depth - without_bn_depth)/without_bn_depth)*100
        
        print(f"  {depth.capitalize()} CNN:")
        print(f"    With BN: {with_bn_depth:.4f}")
        print(f"    Without BN: {without_bn_depth:.4f}")
        print(f"    Improvement: {improvement:.2f}%")
    
    # Training time impact
    print(f"\nTraining Time Impact:")
    print(f"  With BatchNorm - Avg Time: {with_bn['Training_Time'].mean():.1f}s")
    print(f"  Without BatchNorm - Avg Time: {without_bn['Training_Time'].mean():.1f}s")
    time_diff = ((with_bn['Training_Time'].mean() - without_bn['Training_Time'].mean())/without_bn['Training_Time'].mean())*100
    print(f"  Time Increase: {time_diff:.1f}%")


### Activation function and kernel size analysis

In [51]:
def analyze_activation_and_kernels(results):
    """Analyze activation functions and kernel sizes"""
    df = pd.DataFrame(results)
    df['Depth'] = df['Model'].str.split('_').str[0]
    df['Kernel_Size'] = df['Model'].str.extract(r'k(\d+)')[0].astype(int)
    df['Activation'] = df['Model'].str.split('_').str[-1]
    
    print("="*80)
    print("ACTIVATION FUNCTION ANALYSIS")
    print("="*80)
    
    # Overall comparison
    relu_results = df[df['Activation'] == 'relu']
    leaky_relu_results = df[df['Activation'] == 'leaky']
    
    print(f"Overall Performance:")
    print(f"  ReLU - Avg Accuracy: {relu_results['Test_Accuracy'].mean():.4f}")
    print(f"  LeakyReLU - Avg Accuracy: {leaky_relu_results['Test_Accuracy'].mean():.4f}")
    
    print(f"\nTraining Speed:")
    print(f"  ReLU - Avg Time: {relu_results['Training_Time'].mean():.1f}s")
    print(f"  LeakyReLU - Avg Time: {leaky_relu_results['Training_Time'].mean():.1f}s")
    
    print("="*80)
    print("KERNEL SIZE ANALYSIS")
    print("="*80)
    
    # Kernel size comparison
    k3_results = df[df['Kernel_Size'] == 3]
    k5_results = df[df['Kernel_Size'] == 5]
    
    print(f"Overall Performance:")
    print(f"  3x3 Kernels - Avg Accuracy: {k3_results['Test_Accuracy'].mean():.4f}")
    print(f"  5x5 Kernels - Avg Accuracy: {k5_results['Test_Accuracy'].mean():.4f}")
    
    print(f"\nTraining Speed:")
    print(f"  3x3 Kernels - Avg Time: {k3_results['Training_Time'].mean():.1f}s")
    print(f"  5x5 Kernels - Avg Time: {k5_results['Training_Time'].mean():.1f}s")
    
    print(f"\nParameter Count:")
    print(f"  3x3 Kernels - Avg Params: {k3_results['Parameters'].mean():,.0f}")
    print(f"  5x5 Kernels - Avg Params: {k5_results['Parameters'].mean():,.0f}")


### Visualization

In [52]:
def plot_comprehensive_comparison(results):
    """Create comprehensive comparison plots"""
    df = pd.DataFrame(results)
    
    # Parse model components
    df['Depth'] = df['Model'].str.split('_').str[0]
    df['Kernel_Size'] = df['Model'].str.extract(r'k(\d+)')[0].astype(int)
    df['Batch_Norm'] = df['Model'].str.contains('_bn_')
    df['Activation'] = df['Model'].str.split('_').str[-1]
    
    # Create comprehensive comparison plots
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # 1. Accuracy vs Model Depth
    depth_comparison = df.groupby(['Depth', 'Batch_Norm']).agg({
        'Test_Accuracy': 'mean',
        'Parameters': 'mean'
    }).reset_index()
    
    ax = axes[0, 0]
    shallow_bn = depth_comparison[(depth_comparison['Depth'] == 'shallow') & (depth_comparison['Batch_Norm'] == True)]['Test_Accuracy'].values[0]
    shallow_no_bn = depth_comparison[(depth_comparison['Depth'] == 'shallow') & (depth_comparison['Batch_Norm'] == False)]['Test_Accuracy'].values[0]
    deep_bn = depth_comparison[(depth_comparison['Depth'] == 'deep') & (depth_comparison['Batch_Norm'] == True)]['Test_Accuracy'].values[0]
    deep_no_bn = depth_comparison[(depth_comparison['Depth'] == 'deep') & (depth_comparison['Batch_Norm'] == False)]['Test_Accuracy'].values[0]
    
    x = np.arange(2)
    width = 0.35
    ax.bar(x - width/2, [shallow_no_bn, deep_no_bn], width, label='Without BatchNorm', alpha=0.7)
    ax.bar(x + width/2, [shallow_bn, deep_bn], width, label='With BatchNorm', alpha=0.7)
    ax.set_title('Accuracy vs Model Depth')
    ax.set_ylabel('Test Accuracy')
    ax.set_xticks(x)
    ax.set_xticklabels(['Shallow', 'Deep'])
    ax.legend()
    
    # 2. Parameters vs Depth
    ax = axes[0, 1]
    shallow_params = df[df['Depth'] == 'shallow']['Parameters'].mean()
    deep_params = df[df['Depth'] == 'deep']['Parameters'].mean()
    ax.bar(['Shallow CNN', 'Deep CNN'], [shallow_params, deep_params], 
           color=['lightblue', 'darkblue'])
    ax.set_title('Parameter Count Comparison')
    ax.set_ylabel('Number of Parameters')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M' if x >= 1e6 else f'{x/1e3:.0f}K'))
    
    # 3. Impact of Batch Normalization
    ax = axes[0, 2]
    bn_data = df.groupby(['Batch_Norm', 'Depth'])['Test_Accuracy'].mean().unstack()
    bn_data.plot(kind='bar', ax=ax, color=['lightcoral', 'darkred'])
    ax.set_title('Impact of Batch Normalization')
    ax.set_xlabel('Batch Normalization')
    ax.set_ylabel('Test Accuracy')
    ax.set_xticklabels(['Without BN', 'With BN'], rotation=0)
    ax.legend(title='Architecture')
    
    # 4. Kernel Size Comparison
    ax = axes[1, 0]
    kernel_data = df.groupby(['Kernel_Size', 'Depth'])['Test_Accuracy'].mean().unstack()
    kernel_data.plot(kind='bar', ax=ax, color=['lightgreen', 'darkgreen'])
    ax.set_title('Kernel Size Impact')
    ax.set_xlabel('Kernel Size')
    ax.set_ylabel('Test Accuracy')
    ax.set_xticklabels(['3x3', '5x5'], rotation=0)
    ax.legend(title='Architecture')
    
    # 5. Activation Function Comparison
    ax = axes[1, 1]
    activation_data = df.groupby(['Activation', 'Depth'])['Test_Accuracy'].mean().unstack()
    activation_data.plot(kind='bar', ax=ax, color=['orange', 'darkorange'])
    ax.set_title('Activation Function Impact')
    ax.set_xlabel('Activation Function')
    ax.set_ylabel('Test Accuracy')
    ax.set_xticklabels(['LeakyReLU', 'ReLU'], rotation=0)
    ax.legend(title='Architecture')
    
    # 6. Training Time vs Accuracy
    ax = axes[1, 2]
    colors = ['red' if d == 'deep' else 'blue' for d in df['Depth']]
    scatter = ax.scatter(df['Training_Time'], df['Test_Accuracy'], 
                       c=colors, alpha=0.7, s=100)
    ax.set_xlabel('Training Time (seconds)')
    ax.set_ylabel('Test Accuracy')
    ax.set_title('Training Time vs Accuracy')